# IPO Random Forest Models
Predicting Excess Returns using pre-IPO Data and Random Forest Regressions and Classifications

In [236]:
# Library imports
import psycopg2
import os
import pandas as pd
import numpy as np
import requests
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import KBinsDiscretizer
import plotly.graph_objects as go
import plotly.figure_factory as ff

## Gathering data

In [237]:
# Establish connection to PostgreSQL
conn = psycopg2.connect(os.environ.get('DB_CONNECTION_STRING'))

In [238]:
# pre-IPO data
query = 'SELECT DISTINCT ON ("companyName") * FROM ipos ORDER BY "companyName", "createdAt" DESC NULLS LAST;'
ipos = pd.read_sql(query, conn)
ipos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188 entries, 0 to 187
Data columns (total 39 columns):
 #   Column                  Non-Null Count  Dtype              
---  ------                  --------------  -----              
 0   id                      188 non-null    int64              
 1   symbol                  188 non-null    object             
 2   companyName             188 non-null    object             
 3   expectedDate            188 non-null    datetime64[ns, UTC]
 4   auditor                 188 non-null    object             
 5   market                  188 non-null    object             
 6   cik                     188 non-null    object             
 7   address                 188 non-null    object             
 8   city                    188 non-null    object             
 9   state                   188 non-null    object             
 10  zip                     188 non-null    object             
 11  phone                   188 non-null    objec

In [239]:
# Company data
query = f'SELECT * FROM companies ORDER BY "companyName";'
companies = pd.read_sql(query, conn)
companies.iloc[0]

id                                                              369
symbol                                                          TXG
companyName                                      10X Genomics, Inc.
exchange                                                     NASDAQ
industry                                              Biotechnology
website                                  http://www.10xgenomics.com
description       10X Genomics, Inc. is a life science technolog...
CEO                                                   Serge Saxonov
securityName                               10x Genomics Inc Class A
issueType                                                        cs
sector                                            Health Technology
primarySicCode                                                 2836
employees                                                       584
address                                   6230 Stoneridge Mall Road
address2                                        

In [240]:
# Company price data
date_string = "'2019-07-22'"
companies_ids = tuple(companies['id'].to_numpy())
query = f'SELECT * FROM prices WHERE "companyId" IN {companies_ids} AND "date" >= {date_string} ORDER BY "companyId", "date";'
prices = pd.read_sql(query, conn)
prices['date'] = pd.to_datetime(prices['date']) 

In [241]:
# Benchmark price data
r = requests.get(os.environ.get('BENCHMARK_QUERY_STRING'))
raw_benchmark_data = r.json()

In [242]:
benchmark = pd.DataFrame.from_dict(raw_benchmark_data)
benchmark['date'] = pd.to_datetime(benchmark['date']) 

## Build Labeled Series

In [243]:
# 1. Create 21-trading day return series for SPY (average # of trading days per month)
# 2. Extract return for each IPO and its excess return to SPY over the same period
#    this should be a function that takes a trading day size and benchmark return series
# 3. Combine returns with pre-IPO data
# 4. Remove companies that do not have return data available given trading window size
# 5. Convert categorical data to one-hot or binary encoding
# 6. Finialze sample data (X, Y)

In [244]:
# Series variables
price_column = 'close'
period = 21
returns_groupby = 'companyId'

In [245]:
# Create a period return series
def period_return(df, period, price_column, return_type='discrete', zero_idx=True):
    '''
    Create a period return series from DataFrame (requires ASC order).
    '''
    
    types = ('discrete', 'continuous')
    if return_type not in types:
        raise TypeError(f'return_type needs to be of type: {types}')

    zero_period = period
    if zero_idx == True:
        zero_period = period - 1

    if return_type == 'discrete':
        return (df[price_column][zero_period:] - df[price_column][:-zero_period].values) / df[price_column][:-zero_period].values
    else:
        return np.log(df[price_column][zero_period:]) - np.log(df[price_column][:-zero_period].values)

In [246]:
# 1. Benchmark Returns
benchmark['returns21d'] = period_return(benchmark, period, price_column)

In [247]:
# 2. IPO returns
def add_returns_groupby(df, groupby, price_column, period, returns_column='returns'):
    '''
    Adds a return column to original DataFrame. Calculations are done arounding to groupby key.
    
    :returns: Original DataFrame with new column.
    '''
    grouped = df.groupby(groupby)
    returns_series = []
    first_series = None
    for _, group in grouped:
        if first_series is None:
            first_series = period_return(group, period, price_column).rename(returns_column)
        else:
            returns_series.append(period_return(group, period, price_column).rename(returns_column))

    return df.join(first_series.append(returns_series))

In [248]:
# 2. IPO returns
prices = add_returns_groupby(prices, returns_groupby, price_column, period)

In [249]:
# Excess returns
prices = prices.merge(benchmark[['date', 'returns21d']])
prices['ex_returns'] = prices['returns'] - prices['returns21d']

In [250]:
# Match sybmol to id
prices = prices.merge(companies[['id', 'symbol']], left_on='companyId', right_on='id')
prices

,id_x,date,high,low,volume,open,close,uHigh,uLow,uVolume,...,change,changePercent,createdAt,updatedAt,companyId,returns,returns21d,ex_returns,id_y,symbol
0,20936,2019-08-01,13.22,7.66,9314399.0,13.01,8.48,13.22,7.66,9314399.0,...,0.00,0.0000,2020-02-06 23:34:00.006000+00:00,2020-02-06 23:34:00.006000+00:00,332,NaN,0.168240,NaN,332,SNDL
1,20937,2019-08-02,10.48,8.42,2693863.0,8.42,10.45,10.48,8.42,2693863.0,...,1.97,23.2311,2020-02-06 23:34:00.006000+00:00,2020-02-06 23:34:00.006000+00:00,332,NaN,0.182069,NaN,332,SNDL
2,20938,2019-08-05,11.82,10.47,2206717.0,10.69,11.70,11.82,10.47,2206717.0,...,1.25,11.9617,2020-02-06 23:34:00.006000+00:00,2020-02-06 23:34:00.006000+00:00,332,NaN,0.107545,NaN,332,SNDL
3,20939,2019-08-06,13.21,11.99,2180774.0,12.00,13.00,13.21,11.99,2180774.0,...,1.30,11.1111,2020-02-06 23:34:00.006000+00:00,2020-02-06 23:34:00.006000+00:00,332,NaN,0.097477,NaN,332,SNDL
4,20940,2019-08-07,13.22,12.20,1611203.0,13.05,12.85,13.22,12.20,1611203.0,...,-0.15,-1.1538,2020-02-06 23:34:00.006000+00:00,2020-02-06 23:34:00.006000+00:00,332,NaN,0.113691,NaN,332,SNDL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15821,31903,2020-07-06,96.51,73.11,13467931.0,73.39,81.19,96.51,73.11,13467931.0,...,11.78,16.9700,2020-07-07 09:30:18.182000+00:00,2020-07-07 09:30:18.182000+00:00,548,NaN,-0.072555,NaN,548,LMND
15822,32029,2020-07-07,89.38,75.00,4602812.0,83.80,78.79,89.38,75.00,4602812.0,...,-2.40,-2.9600,2020-07-08 09:30:11.720000+00:00,2020-07-08 09:30:11.720000+00:00,548,NaN,-0.099618,NaN,548,LMND
15823,32155,2020-07-08,79.39,68.06,3499175.0,79.00,68.51,79.39,68.06,3499175.0,...,-10.28,-13.0500,2020-07-09 09:30:12.058000+00:00,2020-07-09 09:30:12.058000+00:00,548,NaN,-0.014198,NaN,548,LMND
15824,32282,2020-07-09,79.91,69.03,4178671.0,73.97,77.01,79.91,69.03,4178671.0,...,8.50,12.4100,2020-07-10 09:30:14.207000+00:00,2020-07-10 09:30:14.207000+00:00,548,NaN,0.016505,NaN,548,LMND


In [251]:
# 3/4. Combine the first return observation with pre-IPO data
def combine_return_data(prices, ipos, return_col):
    '''
    Finds a single return to match to the pre-ipo data
    '''
    grouped = prices.groupby('symbol')
    returns_series = []
    for symbol, group in grouped:
        temp_return = group[~pd.isna(group[return_col])][return_col]
        if len(temp_return) > 0:
            returns_series.append({
                'symbol': symbol,
                'ex_returns': temp_return.iloc[0]
            })
    return ipos.merge(pd.DataFrame.from_dict(returns_series), on='symbol')

In [252]:
ipos = combine_return_data(prices, ipos, 'ex_returns')

In [253]:
ipos['ex_returns'].describe()

count    110.000000
mean       0.056870
std        0.447580
min       -0.547960
25%       -0.171290
50%       -0.000421
75%        0.170126
max        3.666887
Name: ex_returns, dtype: float64

In [254]:
fig = ff.create_distplot([ipos['ex_returns']], ['ex_returns'], bin_size=.05, histnorm = '')
fig.show()

## Input Features - Pre-IPO Data Variables
#### Categorical
1. market
2. state
3. industry (v2)
4. sector (v2)

#### Numerical
1. employees
2. priceLow
3. totalExpenses
4. sharesOutstanding
5. revenue
6. netIncome
7. totalAssets
8. totalLiabilities
9. amount
10. percentOffered

In [255]:
def cast_columns_dtype(df, dtype, columns=[], all_columns=True):
    '''
    Casts columns into a specified type.
    '''
    if all_columns == True:
        columns = list(df.columns)

    dtype_dict = {col: dtype for col in columns}
    return df.astype(dtype_dict)
    

def create_labeled_data(data, target_col, categorical=[], numerical=[], retcolumns=False):
    '''
    Create X and y data arrays for machine learning methods.
    '''

    # Numerical
    if len(numerical) > 0:
        X = data[numerical].copy()
    else:
        X = pd.get_dummies(data[categorical])
    # Categorical
    if len(categorical) > 0 and len(numerical):
        X = X.merge(pd.get_dummies(data[categorical]), right_index=True, left_index=True)
    # Cast types
    X = cast_columns_dtype(X, 'float64')

    # Target values
    y = data[target_col].copy()
    
    if retcolumns == True:
        return X.to_numpy(), y.to_numpy().ravel(), list(X.columns)

    return X.to_numpy(), y.to_numpy().ravel()

In [256]:
categorical = ['state', 'market']
numerical = [
    'employees',
    'priceLow',
    'totalExpenses',
    'sharesOutstanding',
    'revenue',
    'netIncome',
    'totalAssets',
    'totalLiabilities',
    'amount',
    'percentOffered'
]
target_col = ['ex_returns']
X, y, X_columns = create_labeled_data(ipos, target_col, categorical, numerical, True)

In [257]:
np.savetxt('X.csv', X, delimiter=',', header=', '.join(X_columns))
np.savetxt('y.csv', y, delimiter=',', header=target_col[0])

## Fitting a Random Forest Model
---

**fit(self, X, y, sample_weight=None)**

Build a forest of trees from the training set (X, y).

|Parameters|Description|
|:--|:--|
|**X : {array-like, sparse matrix} of shape (n_samples, n_features)**|The training input samples.|
|**y : array-like of shape (n_samples,) or (n_samples, n_outputs)**|The target values (class labels in classification, real numbers in regression).|
|**sample_weight : array-like of shape (n_samples,), default=None**|Sample weights. If None, then samples are equally weighted. Splits that would create child nodes with net zero or negative weight are ignored while searching for a split in each node. In the case of classification, splits are also ignored if they would result in any single class carrying a negative weight in either child node.|

In [258]:
# Divide the data in to train, dev sets (70%, 30%)

test_size = 0.30
random_state = 0

X_train, X_dev, y_train, y_dev = train_test_split(X, y, test_size=test_size, random_state=random_state)

### Initial Regression
Fit our initial version of the random forest regression and full list of input variables. This is to see if there is some explanatory or predictive power of our inputs and our target variable--one-month excess market returns.

In [259]:
# 1. 1st try - Default Hyperparameters
regr = RandomForestRegressor(max_depth=2, random_state=0)
regr.fit(X_train, y_train)
train_scores = {}
dev_scores = {}
train_scores['default'] = regr.score(X_train, y_train)
dev_scores['default'] = regr.score(X_dev, y_dev)

# Results
print('R^2 Scores - Default Hyperparameters')
print('Train score:', '{:.4f}'.format(train_scores['default']))
print('Dev score:', '{:.4f}'.format(dev_scores['default']))

# fig = go.Figure(data=[go.Table(header=dict(values=['Data', 'R^2']), \
#                                cells=dict(values=[['train_score', 'dev_score'], [train_score, dev_score]]))])
# fig.update_layout(width=500, height=300, \
#                      margin=dict(l=50, r=50, b=20, t=20, pad=0))
# fig.show()
# y_pred = regr.predict(X_test)
# print(y_pred[0])
# print(y_test[0])

R^2 Scores - Default Hyperparameters
Train score: 0.6262
Dev score: 0.0177


**Horrible first regression!** We have pretty low training performance and out of sample performance is only slightly better than random guessing. Our goal now is to increase performance across the board and see if we are able to find some predictive power from our input variable. Some of the steps are going to try are:
- Train using full depth trees
- Simplify the set of input variables
- Setting `max_features` to a lower value
- Modify RF class to replace standard bootstrapping with sequential bootstrapping


In [260]:
# 2. Train using full depth trees
regr = RandomForestRegressor(max_depth=None, random_state=0)
regr.fit(X_train, y_train)
train_scores['fullDepth'] = regr.score(X_train, y_train)
dev_scores['fullDepth'] = regr.score(X_dev, y_dev)

# Results
print('R^2 Scores - Full Depth Trees')
print('Train score:', '{:.4f}'.format(train_scores['fullDepth']))
print('Dev score:', '{:.4f}'.format(dev_scores['fullDepth']))

R^2 Scores - Full Depth Trees
Train score: 0.8095
Dev score: 0.0416


Large improvement in training score, but no change in dev score. Model is not able to generalize to out of the training data.

In [261]:
# 3. Increase number of trees to 10,000 (versus 100 default)
regr = RandomForestRegressor(max_depth=None, random_state=0, n_estimators=10000)
regr.fit(X_train, y_train)
train_scores['fullDepth'] = regr.score(X_train, y_train)
dev_scores['fullDepth'] = regr.score(X_dev, y_dev)

# Results
print('R^2 Scores - 10,000 Trees')
print('Train score:', '{:.4f}'.format(train_scores['fullDepth']))
print('Dev score:', '{:.4f}'.format(dev_scores['fullDepth']))

# TODO: Need to F-test dev score

R^2 Scores - 10,000 Trees
Train score: 0.8338
Dev score: 0.0055


In [262]:
# 4. Lower max_features than n_features
regr = RandomForestRegressor(max_depth=None, random_state=0, n_estimators=10000, max_features='sqrt')
regr.fit(X_train, y_train)
train_scores['sqrtMaxF'] = regr.score(X_train, y_train)
dev_scores['sqrtMaxF'] = regr.score(X_dev, y_dev)

# Results
print('R^2 Scores - Sqrt Max Features')
print('Train score:', '{:.4f}'.format(train_scores['sqrtMaxF']))
print('Dev score:', '{:.4f}'.format(dev_scores['sqrtMaxF']))

R^2 Scores - Sqrt Max Features
Train score: 0.8460
Dev score: -0.0992


## Random Forest Classification
---
### Trying a simpler problem
Instead of trying to predict the specific excess return values, we are going to bucket the returns and try to fit a random forest classification model to predict the bucket assignment. With this simpler problem we are hoping to find some precitive power of our model.

In [263]:
# 1. Binary classification of target values: positive versus negative returns
ipos2 = ipos.copy()
ipos2['ex_returns'] = (ipos2['ex_returns'] > 0).astype(int)

In [264]:
# Divide the data in to train, dev sets (70%, 30%)
X, y, X_columns = create_labeled_data(ipos2, target_col, categorical, numerical, True)

test_size = 0.30
random_state = 0

X_train, X_dev, y_train, y_dev = train_test_split(X, y, test_size=test_size, random_state=random_state)

In [265]:
train_scores_clf = {}
dev_scores_clf = {}

# Model
clf = RandomForestClassifier(max_depth=None, random_state=0, n_estimators=10000)
clf.fit(X_train, y_train)
train_scores_clf['maxDepth'] = clf.score(X_train, y_train)
dev_scores_clf['maxDepth'] = clf.score(X_dev, y_dev)

# Results
print('Classification Scores - Full Depth Trees')
print('Train score:', '{:.4f}'.format(train_scores_clf['maxDepth']))
print('Dev score:', '{:.4f}'.format(dev_scores_clf['maxDepth']))

Classification Scores - Full Depth Trees
Train score: 1.0000
Dev score: 0.5758


In [266]:
# 2. Binning of target values using uniform

bins = [-1.0001, -0.75, -0.5, -0.25, 0, 0.25, 0.5, 0.75, 1, 1000]
labels = [i for i in range(0, 9)]
ipos['bin_ex_returns'] = pd.cut(ipos['ex_returns'], bins=bins, labels=labels)

X, y, X_columns = create_labeled_data(ipos, 'bin_ex_returns', categorical, numerical, True)

In [267]:
# Divide the data in to train, dev sets (70%, 30%)
test_size = 0.30
random_state = 0

X_train, X_dev, y_train, y_dev = train_test_split(X, y, test_size=test_size, random_state=random_state)

In [268]:
# Model
clf = RandomForestClassifier(max_depth=None, random_state=0, n_estimators=10000)
clf.fit(X_train, y_train)
train_scores_clf['maxDepth'] = clf.score(X_train, y_train)
dev_scores_clf['maxDepth'] = clf.score(X_dev, y_dev)

# Results
print('Classification Scores - Full Depth Trees')
print('Train score:', '{:.4f}'.format(train_scores_clf['maxDepth']))
print('Dev score:', '{:.4f}'.format(dev_scores_clf['maxDepth']))

Classification Scores - Full Depth Trees
Train score: 1.0000
Dev score: 0.2424


## Testing the Final Model
---

## Logistic Regression for Comparison

In [269]:
# Model
clf = LogisticRegression(solver='liblinear', random_state=random_state)
clf.fit(X_train, y_train)
train_scores_clf['logit'] = clf.score(X_train, y_train)
dev_scores_clf['logit'] = clf.score(X_dev, y_dev)

# Results
print('Classification Scores - Logit Regression')
print('Train score:', '{:.4f}'.format(train_scores_clf['logit']))
print('Dev score:', '{:.4f}'.format(dev_scores_clf['logit']))


Classification Scores - Logit Regression
Train score: 0.4286
Dev score: 0.3030
